# Deep Neural Networks from scratch:

We'll be implementing a neural network to recognize handwritten digits. We'll be using the same MNIST as we had done in the previous section.

To recognize the digits we will use a 3-layer neural network. The input layer of the network contains neurons encoding the values of the input pixels. As discussed earlier, our training data will consist of many 28\*28 pixel images of scanned hand-written digits, and so the input layer contains 784 = 28\*28 neurons. The input pixels are greyscale, with a value of 0 representing white and a value of 1 representing black, and in between values representing gradually darkening shades of grey.

The second layer of the network is a hidden layer.

The output layer of the network contains 10 neurons. If the first neuron fires, i.e., has an ouput~1, then that will indicate that the network thinks the digit is a 1. If the second neuron fires then that will indicate that the network thinks the digit is a 1. And so on. A little more precisely we number the neurons from 0 through 9, and figure out which neuron has the highest activation value.

In this network we will be applying <b>sigmoid</b> activation on output from each neuron in the hidden layer and output layer and finally we will be using the <b> mean squared error </b> (MSE) to calculate the error.

In [100]:
import random
import numpy as np
import random

In [101]:
'''
The centerpiece is a Network class, which we use to represent a neural network.
'''
class Network(object):
    
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                       for x, y in zip(sizes[:-1], sizes[1:])]
    
    '''
    In this code, the list sizes contains the number of neurons in the respective layers. So, for example, if we want to create 
    a network object with 2 neurons in the first layer, 3 neurons in the second layer, and 1 neuron in the final layer, we'd
    write the following code:
    net = Network([2, 3, 1])
    The biases and weights in the Network object are all initialized randomly, using the np.random.randn function to generate 
    Gaussian distributions with mean 0 and standard deviation 1. This random initialization will give our stochastic gradient
    descent algorithm a place to start from.
    Also, make a note of how the Network initialization code assumes that the first layer of neurons is an input layer, and omits
    to set any biases for those neurons.
    The biases and weights are stored as a list of numpy matrices. So, net.weights[1] is a Numpy matrix storing the weights
    connecting the second and third layer of neurons. net.weights[1] is rather verbose, let's just denote that by w. As mentioned
    in backpropagation theory, it's a matrix such that w[j][k] is the weight for the connection between the kth neuron in the
    second layer, and jth neuron in the third layer.
    ''' 
    
    
    #Now we add a feedforward method to the Network class, which, given an input for the network, returns the corresponding
    #output. It is important to note here that the input a is an (n, 1) Numpy ndarray, not a (n,) vector. Here n is the number
    #of inputs to the vector. If we try to use (n,) vector as input we'll get strange results. Although using an (n,) vector 
    #appears the more natural choice, using an (n, 1) ndarray makes it particularly easy to modify the code to feedforward
    #multiple inputs at once, and that is sometimes convenient.
    def feedforward(self, a):
        '''Return the output of the network if "a" is input.'''
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    
    #The main thing we want our network objects to do is to learn. So, we'll write an SGD method which implements Stochastic
    #Gradient Descent.
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None):
        '''
        Train the neural network using mini-batch stochastic gradient descent. The "training data" is a list
        of tuples "(x, y)" representing the training inputs and the desired outputs. The other non-optional
        parameters are self-explanatory. If "test_data" is provided then the network will be evaluated against
        the test data after each epoch, and partial progress printed out. This is useful for tracking progress
        but slows things down substantially.
        '''
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                           for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))
    '''
    The training data is a list of tuples (x, y) representing the inputs and corresponding desired outputs. The variables,
    epochs and mini_batch_size are the number of epochs to train for, and the size of mini batches to use when sampling.
    eta is the learning rate.
    
    The code works as follows. In each epoch, it starts by randomly shuffling the training data, and then partitions it into
    mini-batches of appropriate size. This is an easy way of sampling randomly from training data. Then for each mini-batch 
    we apply a single step of gradient descent. This is done by the code self.update_mini_batch(mini_batch, eta) which updates
    weights and biases according to a single iteration of gradient descent, using just the training data in mini_batch. Here's
    the code for update_mini_batch method:
    '''
    
    def update_mini_batch(self, mini_batch, eta):
        '''
        Update the network's weights and biases by applying gradient descent using backpropagation to a single mini batch.
        The mini_batch is a list of tuples "(x, y)", and "eta" is the learning rate.
        '''
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta/len(mini_batch))*nw
                       for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch))*nb
                      for b, nb in zip(self.biases, nabla_b)]
        
    '''
    Most of the work in the above code is done by: "delta_nabla_b, delta_nabla_w = self.backprop(x, y)".
    This invokes the backpropagation algorithm, which is a fast way of computing the gradient of the cost function.
    As discussed in the backpropagation theory section, given a mini-bacth of m training examples, the following algorithm
    applies a gradient descent learning step based on that mini-batch.
    1. Input a set of training examples
    2. For each training example x: Set the corresponding input activation a(1)[x] and perform the following steps:
        Feedforward: For each l = 2,3,...,L compute z(l)[x] = w(l)a(l-1)[x] + b(l) and a(l)[x] = sigmoid(z(l)[x]).
        Output Error: Compute the vector E(l)[x] = (dC/da(l)[x])*sigmoid_prime(z(l)[x]).
        Backpropagate the error: for each l = L-1, L-2,...,2 compute  E(l) = ((w(l+1).T).E(l+1))*sigmoid_prime(z(l)).
    '''
    
    def backprop(self, x, y):
        '''
        Return a tuple "(nabla_b, nabla_w)" representing the gradient for the cost function C_x. "nabla_b" and "nabla_w"
        are layer-by-layer lists of numpy arrays, similar to "self.biases" and "self.weights"
        '''
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] #list to store all the activations, layer by layer
        zs = [] #list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1],y)*sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        #Note that the variable l in the loop below is used a little differently. Here, l = 1 means the last layer of neurons
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        '''
        Return the number of test inputs for which neural network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever neuron in the final layer has the highest activation
        '''
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        '''
        Returns a vector of partial derivatives
        '''
        return (output_activations - y)
    
#Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Now we'll write functions to load the dataset. You can download the data from git by running the command:
'git clone https://github.com/mnielsen/neural-networks-and-deep-learning.git'

In [102]:
'''
mnist_loader
------------

A library to load the MNIST image data. For details of the data structure that are returned, see the doc strings for
"load_data" and "load_data_wrapper". In practice, "load_data_wrapper" is the function usually called by our neural network
code.
'''

#Importing the libraries
import _pickle as cPickle
import gzip
import numpy as np

def load_data():
    '''
    Return the MNIST data as a tuple containing the training data, the validation data, and the test data.
    
    The "training data" is returned as a tuple with 2 entries. The first entry contains the actual training images. This
    is a numpy ndarray with 50,000 entries. Each entry is, in turn, a numpy ndarray with 784 values, representing the 
    28*28 = 784 pixels in a single MNIST image.
    
    The second entry in the "training data" tuple is a numpy ndarray containing 50,000 entries. Those entries are just the
    digit values (0...9) for the corresponding images contained in the first entry of the tuple.
    
    The "validation data" and "test data" are similar, except each contains only 10,000 images.
    
    This is a nice data format, but for use in neural networks it's helpful to modify the format of the "training data" a
    little. That's done in the wrapper function "load_data_wrapper", see below.
    '''
    f = gzip.open('neural-networks-and-deep-learning/data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding = 'latin1')
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    '''
    Return a tuple containing "(training_data, validation_data, test_data)". Based on "load_data", but the format is more
    convenient for use in our implementation of neural networks.
    
    In particular, "training_data" is a list containing 50,000 2-tuples "(x, y)". "x" is a 784-dimensional numpy.ndarray
    containing the input image. "y" is a 10-dimensional numpy.ndarray representing the unit vector corresponding to the
    correct digit for "x".
    
    "validation_data" and "test_data" are lists containing 10,000 2-tuples "(x, y)". In each case "x" is a 784-dimensional
    numpy.ndarray containing the input image, and "y" is the corresponding classification, i.e., the digit values (integers)
    corresponding to "x".
    
    Obviously, this means we are using slightly different formats for the training data and the validation / test data. These
    formats turn out to be the most convenient for use in our neural network code.
    '''
    tr_d, va_d, te_d = load_data()
    training_data = [(np.reshape(x, (784, 1)), vectorized_result(y)) for x, y in zip(tr_d[0], tr_d[1])]
    validation_data = [(np.reshape(x, (784, 1)), y) for x, y in zip(va_d[0], va_d[1])]
    test_data = [(np.reshape(x, (784, 1)), y) for x, y in zip(te_d[0], te_d[1])]
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    '''
    Return a 10-dimensional unit vector with a 1.0 in the jth position and zeroes elsewhere. This is used to convert a digit
    (0...9) into a corresponding desired output from the neural network.
    '''
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

Training the model:

In [103]:
#loading the data
training_data, validation_data, test_data = load_data_wrapper()

In [104]:
#defining the network object
net = Network([784, 30, 10])

#executing the training loop
net.SGD(training_data, 30, 10, 3.0, test_data = test_data)

Epoch 0: 9010 / 10000
Epoch 1: 9204 / 10000
Epoch 2: 9316 / 10000
Epoch 3: 9372 / 10000
Epoch 4: 9399 / 10000
Epoch 5: 9420 / 10000
Epoch 6: 9408 / 10000
Epoch 7: 9456 / 10000
Epoch 8: 9457 / 10000
Epoch 9: 9423 / 10000
Epoch 10: 9473 / 10000
Epoch 11: 9461 / 10000
Epoch 12: 9441 / 10000
Epoch 13: 9473 / 10000
Epoch 14: 9472 / 10000
Epoch 15: 9496 / 10000
Epoch 16: 9493 / 10000
Epoch 17: 9490 / 10000
Epoch 18: 9500 / 10000
Epoch 19: 9475 / 10000
Epoch 20: 9491 / 10000
Epoch 21: 9498 / 10000
Epoch 22: 9469 / 10000
Epoch 23: 9481 / 10000
Epoch 24: 9479 / 10000
Epoch 25: 9497 / 10000
Epoch 26: 9501 / 10000
Epoch 27: 9482 / 10000
Epoch 28: 9486 / 10000
Epoch 29: 9494 / 10000


As we can see we are able to achieve 95% accuracy.